In [1]:
import numpy as np
import json
import os
import pynbody

In [2]:
# The simulation data is made of 1000s of unformatted fortran binaries 
# (not very useful I know, but these codes are primarily optimized for HPC performance rather than user convenience).
# The pynbody package (which you might need to pip install) will make sense of the data formart
# and allow you to convert it to data tables suiting your need

#whole_simulation = pynbody.load("./output_00186/")

In [3]:
# The simulation itself is a large chunk of the Universe, with its large scale structure and cosmological filamentary structure
# You are welcome to explore it, but for the purpose of this simulation, we actually focussed all the computational power
# to produce a very well-resolved Milky Way galaxy in the centre
# I am now just centering on it and extracting its direct surroundings
#pynbody.analysis.halo.center(whole_simulation.halos()[1])

In [4]:
#galaxy = whole_simulation[pynbody.filt.Sphere(pynbody.array.SimArray(400, units='kpc'))]

# Explore our chosen object

In [5]:
# This galaxy contains a certain number of star particles 
# In the context of these simulations (using smoothed particle hydrodynamics), you can view the word "particle"
# as equivalent to the simulation's "pixel" or resolution element. More particles == better-resolved simulations
#galaxy.star

In [6]:
# Gas particles
#galaxy.gas

In [7]:
# And dark matter particles
#galaxy.dm

In [8]:
# The data is organised into families. 
#print(f"Here are all the families we are working with: {galaxy.families()}.")
#print(f"Here are the keys for dark matter: {galaxy.dm.keys()}.")
#print(f"Here are the keys for stars: {galaxy.star.keys()}.")
#print(f"Here are the keys for gas particles: {galaxy.gas.keys()}.")
#gas_density = galaxy.gas['rho']  # Gas density is a field that we have.
#gas_temp = galaxy.gas['temp']  # Gas temperature is not one of the fields listed, however we can still refer to temperature, and pynbody will simply calculate it using physics!

In [9]:
# For each family of particles, I can ask for their individual physical quantities
# for example 'mass', 'x', 'y', 'z', 'vx', 'vy', 'vz' are accessible across all families
#galaxy.star['mass'].in_units("Msol")

In [10]:
#galaxy.gas['mass'].in_units("g")

In [11]:
#galaxy.dm['mass'].in_units("kg")

In [12]:
# But not all physical quantities apply to all families
# The mass fraction of ozygen, for example, only applies to stars and gas
# (dark matter does not interact with visible matter like iron)
#galaxy.star['metal_ox']

In [13]:
# You can all available fields for each family with
#galaxy.star.keys()

In [14]:
#galaxy.dm.keys()

# Actual Code:

In [ ]:
# ==========================================
# CONFIGURATION
# ==========================================

export_config = {
    "gas": {
        "family_selector": lambda g: g.gas,
        "attributes": [
            # Derived by pynbody from 'p' and 'rho' (We will try to get it)
            {"name": "temp", "pynbody_name": "temp", "units": "K", "log": True}, 
            # 'rho' is in your keys list
            {"name": "density", "pynbody_name": "rho", "units": "g cm^-3", "log": True},
            # 'mass' is in your keys list
            {"name": "mass", "pynbody_name": "mass", "units": "Msol", "log": True},
            # 'metal_ox' is in your keys list
            {"name": "oxygen", "pynbody_name": "metal_ox", "units": None, "log": False},
            # 'smooth' is in your keys list (Critical for visualization size)
            {"name": "smooth", "pynbody_name": "smooth", "units": "kpc", "log": False}
        ]
    },
    "star": {
        "family_selector": lambda g: g.star,
        "attributes": [
            {"name": "mass", "pynbody_name": "mass", "units": "Msol", "log": True},
            # We know this exists from your first prompt, even if keys() didn't show it
            {"name": "oxygen", "pynbody_name": "metal_ox", "units": None, "log": False},
            # We will TRY to derive age. If tform is missing, this will skip.
            {"name": "age", "pynbody_name": "age", "units": "Gyr", "log": False}
        ]
    },
    "dm": {
        "family_selector": lambda g: g.dm,
        "attributes": [
            {"name": "mass", "pynbody_name": "mass", "units": "Msol", "log": True}
        ]
    }
}

OUTPUT_DIR = "./GalaxyExport"

# ==========================================
# PROCESSING FUNCTIONS
# ==========================================

def prepare_simulation(sim_obj):
    """
    1. Finds the main halo (Halo 1).
    2. Centers the simulation on it.
    3. Filters for a 400kpc sphere.
    4. Rotates it face-on.
    """
    print("1. Identifying Main Halo (Halo 1)...")
    # We load halo 1 explicitly. This is the main galaxy.
    h = sim_obj.halos()[1]
    
    print("2. Centering simulation on Halo 1...")
    pynbody.analysis.halo.center(h)
    
    print("3. Filtering: Cutting out 400 kpc sphere...")
    # We apply the sphere filter to the halo, creating our galaxy object
    galaxy_sphere = h[pynbody.filt.Sphere(pynbody.array.SimArray(400, units='kpc'))]
    
    print("4. Rotating Face-on...")
    pynbody.analysis.angmom.faceon(galaxy_sphere)
    
    return galaxy_sphere

def export_data(galaxy):
    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)
        
    manifest = {}
    sim_radius = 400.0 # kpc
    
    for family_name, config in export_config.items():
        print(f"\n--- Processing {family_name.upper()} ---")
        
        # 1. Select Particles
        particles = config["family_selector"](galaxy)
        count = len(particles)
        
        if count == 0:
            print(f"Skipping {family_name} (0 particles)")
            continue
            
        family_dir = os.path.join(OUTPUT_DIR, family_name)
        if not os.path.exists(family_dir):
            os.makedirs(family_dir)
            
        family_manifest = {
            "count": count,
            "position_file": f"{family_name}/positions.bin",
            "attributes": []
        }

        # 2. Export Positions
        print(f"  Exporting {count} positions...")
        # Get positions in kpc
        pos_data = particles['pos'].in_units('kpc')
        
        # Normalize to -0.5 to 0.5 range for Unity
        pos_norm = (pos_data / (sim_radius * 2)).astype(np.float32)
        
        # Unity Conversion: Swap Z and Y so galaxy is flat on floor
        # Sim(x, y, z) -> Unity(x, z, y)
        pos_unity = np.zeros_like(pos_norm)
        pos_unity[:, 0] = pos_norm[:, 0]
        pos_unity[:, 1] = pos_norm[:, 2] # z becomes y
        pos_unity[:, 2] = pos_norm[:, 1] # y becomes z

        # GPU wants 4-component vectors, so pad with 1.0
        pos_padded = np.column_stack((pos_unity, np.ones(pos_unity.shape[0], dtype=np.float32)))
        
        pos_padded.tofile(os.path.join(family_dir, "positions.bin"))
        
        # 3. Export Attributes
        for attr in config["attributes"]:
            attr_name = attr["name"]
            pyn_name = attr["pynbody_name"]
            
            print(f"  Attempting export: {attr_name} ({pyn_name})...", end="")
            
            try:
                # Try to access the array (this triggers derivation if needed)
                data_array = particles[pyn_name]
                
                # Convert units
                if attr["units"]:
                    data_array = data_array.in_units(attr["units"])
                
                # Convert to numpy float32
                arr = np.array(data_array, dtype=np.float32)
                
                # Log Scale Logic
                is_log = attr["log"]
                if is_log:
                    # distinct tiny value to avoid log(0) errors
                    arr[arr <= 0] = 1e-10 
                    arr = np.log10(arr)
                
                d_min = float(np.min(arr))
                d_max = float(np.max(arr))
                
                # Save
                filename = f"{attr_name}.bin"
                arr.tofile(os.path.join(family_dir, filename))
                
                print(f" Success! (min: {d_min:.2f}, max: {d_max:.2f})")
                
                # Add to manifest
                family_manifest["attributes"].append({
                    "name": attr_name,
                    "file": f"{family_name}/{filename}",
                    "min": d_min,
                    "max": d_max,
                    "is_log": is_log,
                    "units": attr["units"] if attr["units"] else "dimensionless"
                })
                
            except KeyError:
                print(f" FAILED. '{pyn_name}' not found or calculable.")
            except Exception as e:
                print(f" ERROR: {e}")

        manifest[family_name] = family_manifest

    # 4. Write Manifest
    with open(os.path.join(OUTPUT_DIR, "manifest.json"), "w") as f:
        json.dump(manifest, f, indent=4)
    
    print("\n--- Export Complete ---")

# ==========================================
# EXECUTE
# ==========================================
# Run the pipeline
# Note: 'whole_simulation' must be loaded from your initial cells
whole_simulation = pynbody.load("./output_00186/")
processed_galaxy = prepare_simulation(whole_simulation)
export_data(processed_galaxy)

OSError: File WindowsPath('output_00186'): format not understood or does not exist